# Homework 1

## Description

### Data
[Individual household electric power consumption dataset](https://archive.ics.uci.edu/ml/datasets/individual+household+electric+power+consumption) - about 2 million instances, 20MB (compressed) in size


### Format
One text file consisting of lines of records.

Each record contains 9 attributes separated by semicolons: 
1. date: Date in format dd/mm/yyyy 
2. time: time in format hh:mm:ss 
3. global_active_power: household global minute-averaged active power (in kilowatt) 
4. global_reactive_power: household global minute-averaged reactive power (in kilowatt) 
5. voltage: minute-averaged voltage (in volt) 
6. global_intensity: household global minute-averaged current intensity (in ampere) 
7. sub_metering_1: energy sub-metering No. 1 (in watt-hour of active energy)
It corresponds to the kitchen, containing mainly a dishwasher, an oven and a microwave (hot plates are not electric but gas powered) 
8. sub_metering_2: energy sub-metering No. 2 (in watt-hour of active energy)
It corresponds to the laundry room, containing a washing-machine, a tumble-drier, a refrigerator and a light. 
9. sub_metering_3: energy sub-metering No. 3 (in watt-hour of active energy)
It corresponds to an electric water-heater and an air-conditioner.


### Task
3 subtasks:
+ (30pt) Output the minimum, maximum, and count of the columns: ‘global active power’, ‘global reactive power’, ‘voltage’, and ‘global intensity’
+ (30pt) Output the mean and standard deviation of these columns
+ (40pt) Perform min-max normalization on the columns to generate normalized output

### Implementation Issues
+ Missing values
+ Conversion of data types


## Implementation

In [2]:
// Pre-Configured Spark Context in sc

println("Spark Entity:       " + spark)
println("Spark version:      " + spark.version)
println("Spark master:       " + spark.sparkContext.master)
println("Running 'locally'?: " + spark.sparkContext.isLocal)

Spark Entity:       org.apache.spark.sql.SparkSession@76d1a466
Spark version:      2.3.0
Spark master:       local[*]
Running 'locally'?: true


In [4]:
import java.io.File
val data = spark.sparkContext.textFile("./data/data.txt")
data.take(5).foreach(println)

Date;Time;Global_active_power;Global_reactive_power;Voltage;Global_intensity;Sub_metering_1;Sub_metering_2;Sub_metering_3
16/12/2006;17:24:00;4.216;0.418;234.840;18.400;0.000;1.000;17.000
16/12/2006;17:25:00;5.360;0.436;233.630;23.000;0.000;1.000;16.000
16/12/2006;17:26:00;5.374;0.498;233.290;23.000;0.000;2.000;17.000
16/12/2006;17:27:00;5.388;0.502;233.740;23.000;0.000;1.000;17.000


In [3]:
// Remove Header
val header = data.first
val rows = data.filter(l => l != header)
data.take(5).foreach(println)

Date;Time;Global_active_power;Global_reactive_power;Voltage;Global_intensity;Sub_metering_1;Sub_metering_2;Sub_metering_3
16/12/2006;17:24:00;4.216;0.418;234.840;18.400;0.000;1.000;17.000
16/12/2006;17:25:00;5.360;0.436;233.630;23.000;0.000;1.000;16.000
16/12/2006;17:26:00;5.374;0.498;233.290;23.000;0.000;2.000;17.000
16/12/2006;17:27:00;5.388;0.502;233.740;23.000;0.000;1.000;17.000


In [4]:
val flattenData = rows.
    flatMap{ dataString =>
        dataString.split(";").
            zipWithIndex.
            filter{
                case (value,index) => index >= 2 && index <= 5 && value !="?"
            }.map{
                case (value,index) => 
                    index match{
                        case 2 => ("Global Active Power",value.toDouble)
                        case 3 => ("Global Reactive Power",value.toDouble)
                        case 4 => ("Voltage",value.toDouble)
                        case 5 => ("Global Intensity",value.toDouble)
                    }
                }
    }
flattenData.take(10).foreach(println)

(Global Active Power,4.216)
(Global Reactive Power,0.418)
(Voltage,234.84)
(Global Intensity,18.4)
(Global Active Power,5.36)
(Global Reactive Power,0.436)
(Voltage,233.63)
(Global Intensity,23.0)
(Global Active Power,5.374)
(Global Reactive Power,0.498)


### Task 1 - Find Min, Max and Count

In [5]:
val count = flattenData.map{case (k,v) => (k,1)}.reduceByKey((i, j) => i+j).collectAsMap()
count

Map(Global Active Power -> 2049280, Global Reactive Power -> 2049280, Global Intensity -> 2049280, Voltage -> 2049280)

In [6]:
val max = flattenData.reduceByKey{(i, j) => if (i>j) i else j}.collectAsMap()
max

Map(Global Active Power -> 11.122, Global Reactive Power -> 1.39, Global Intensity -> 48.4, Voltage -> 254.15)

In [7]:
val min = flattenData.reduceByKey{(i, j) => if (i<j) i else j}.collectAsMap()
min

Map(Global Active Power -> 0.076, Global Reactive Power -> 0.0, Global Intensity -> 0.2, Voltage -> 223.2)

### Task 2 - Mean & Standard Deviation

In [8]:
val average = flattenData.reduceByKey((i, j) => i+j).map{case (i, j) => (i,j/2049280)}.collectAsMap()
average

Map(Global Active Power -> 1.0916150365005446, Global Reactive Power -> 0.12371447630388221, Global Intensity -> 4.627759310588324, Voltage -> 240.8398579745135)

In [9]:
val std = flattenData.
    map{case (k,v) => (k, scala.math.pow(v-average(k),2))}.
    reduceByKey((i,j) => i+j).
    map{case (k,v) => (k, math.sqrt(v/count(k)))}.collectAsMap()
std

Map(Global Active Power -> 1.0572939031266613, Global Reactive Power -> 0.11272195204783488, Global Intensity -> 4.444395175407247, Voltage -> 3.239985888491343)

### Task 3 - Min-Max Normalization

In [10]:
val norm = flattenData.take(10).map{case (k,v) => (k, (v-min(k))/(max(k)-min(k)))}
norm

Array((Global Active Power,0.3747963063552418), (Global Reactive Power,0.30071942446043165), (Voltage,0.376090468497577), (Global Intensity,0.37759336099585067), (Global Active Power,0.4783632084012313), (Global Reactive Power,0.31366906474820144), (Voltage,0.33699515347334413), (Global Intensity,0.47302904564315357), (Global Active Power,0.4796306355241717), (Global Reactive Power,0.35827338129496406))

## Result 

In [11]:
object MyFunctions {
    def myprint(s: String): Unit = {
        println("For "+s+":")
        println("        Number of Meaningful Data - " + count(s))
        println("        Maximum Value - " + max(s))
        println("        Minimum Value - " + min(s))
        println("        Mean - " + average(s))
        println("        Standard Deviation - " + std(s) + "\n")
    }    
}

MyFunctions.myprint("Global Active Power")
MyFunctions.myprint("Global Reactive Power")
MyFunctions.myprint("Voltage")
MyFunctions.myprint("Global Intensity")


For Global Active Power:
        Number of Meaningful Data - 2049280
        Maximum Value - 11.122
        Minimum Value - 0.076
        Mean - 1.0916150365005446
        Standard Deviation - 1.0572939031266613

For Global Reactive Power:
        Number of Meaningful Data - 2049280
        Maximum Value - 1.39
        Minimum Value - 0.0
        Mean - 0.12371447630388221
        Standard Deviation - 0.11272195204783488

For Voltage:
        Number of Meaningful Data - 2049280
        Maximum Value - 254.15
        Minimum Value - 223.2
        Mean - 240.8398579745135
        Standard Deviation - 3.239985888491343

For Global Intensity:
        Number of Meaningful Data - 2049280
        Maximum Value - 48.4
        Minimum Value - 0.2
        Mean - 4.627759310588324
        Standard Deviation - 4.444395175407247



In [12]:
// For implicit conversions from RDDs to DataFrames
val spark2 = spark
import spark2.implicits._

object Norm {
    def AP(s: String): Double = {
        process(s, "Global Active Power")
    } 
    
    def RP(s: String): Double = {
        process(s, "Global Reactive Power")
    } 
    
    def V(s: String): Double = {
        process(s, "Voltage")
    } 
    
    def I(s: String): Double = {
        process(s, "Global Intensity")
    } 
    
    def process(s: String, k: String): Double = {
        var retVal = 0.0
        if(s == "?"){
            retVal = Double.NaN
        }else{
            retVal = (s.toDouble-min(k))/(max(k)-min(k))
        }
        retVal
    }
}

val dataDF = rows.
                map(_.split(";")).
                map(att => (att(0), att(1), Norm.AP(att(2)), Norm.RP(att(3)), Norm.V(att(4)), Norm.I(att(5)) )).
                toDF("Date", "Time", "Active_Power", "Reactive_Power","Voltage","Intensity")
                


In [17]:
dataDF.write.csv("./data/output")

In [14]:
dataDF.createOrReplaceTempView("records")
spark.sql("SELECT * FROM records WHERE date = '28/4/2007'").show(25)

+---------+--------+--------------------+-------------------+-------------------+--------------------+
|     Date|    Time|        Active_Power|     Reactive_Power|            Voltage|           Intensity|
+---------+--------+--------------------+-------------------+-------------------+--------------------+
|28/4/2007|00:00:00| 0.11696541734564549| 0.0618705035971223|0.31825525040387775| 0.11618257261410789|
|28/4/2007|00:01:00|  0.1171464783632084| 0.0618705035971223|0.32374798061389354| 0.11618257261410789|
|28/4/2007|00:02:00| 0.11732753938077133|0.06330935251798561| 0.3350565428109854| 0.11618257261410789|
|28/4/2007|00:03:00|  0.1171464783632084| 0.0618705035971223| 0.3295638126009697| 0.11618257261410789|
|28/4/2007|00:04:00| 0.11696541734564549| 0.0618705035971223| 0.3247172859450729| 0.11618257261410789|
|28/4/2007|00:05:00| 0.11696541734564549| 0.0618705035971223|0.32213247172859427| 0.11618257261410789|
|28/4/2007|00:06:00|  0.1171464783632084| 0.0618705035971223|0.3285945072

103820004 Michael Fu